In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr.dataset import load_dataset
from lib.ehr.icu_interface import Inpatients

In [3]:
import logging
logging.root.level = logging.DEBUG


In [4]:
# Assign the folder of the dataset to `DATA_FILE`.

HOME = os.environ.get('HOME')
DATA_DIR = f'{HOME}/GP/ehr-data'
SOURCE_DIR = os.path.abspath("..")

with U.modified_environ(DATA_DIR=DATA_DIR):
    m4inpatient_dataset = load_dataset('M4ICU')
   

DEBUG:root:Loading dataframes
DEBUG:root:[DONE] Loading dataframes


INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


INFO:root:Unrecognised ICD v10 codes: 3323 (28.74%)
DEBUG:root:
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD10'> codes (3323)
                    to be removed: ['E08.3513', 'E10.3213', 'E10.3219', 'E10.3291', 'E10.3293', 'E10.3299', 'E10.3313', 'E10.3319', 'E10.3393', 'E10.3399', 'E10.3411', 'E10.3413', 'E10.3491', 'E10.3511', 'E10.3512', 'E10.3513', 'E10.3519', 'E10.3522', 'E10.3523', 'E10.3531', 'E10.3532', 'E10.3559', 'E10.3591', 'E10.3592', 'E10.3593', 'E10.3599', 'E11.3213', 'E11.3219', 'E11.3291', 'E11.3292', 'E11.3293', 'E11.3299', 'E11.3311', 'E11.3313', 'E11.3319', 'E11.3391', 'E11.3393', 'E11.3399', 'E11.3413', 'E11.3419', 'E11.3491', 'E11.3492', 'E11.3493', 'E11.3499', 'E11.3513', 'E11.3519', 'E11.3521', 'E11.3532', 'E11.3542', 'E11.3553', 'E11.3591', 'E11.3592', 'E11.3593', 'E11.3599', 'H34.8112', 'H34.8120', 'H34.8122', 'H34.8192', 'H34.8310', 'H34.8320', 'H35.3110', 'H35.3120', 'H35.3130', 'H35.3131', 'H35.3190', 'H35.3220', 'H35.3230', 'H35.3290'

INFO:root:Unrecognised ICD v9 codes: 118 (1.63%)
DEBUG:root:
                    Unrecognised <class 'lib.ehr.coding_scheme.DxICD9'> codes (118)
                    to be removed: ['041.49', '173.21', '173.22', '173.30', '173.31', '173.32', '173.40', '173.41', '173.42', '173.50', '173.51', '173.52', '173.59', '173.60', '173.61', '173.62', '173.70', '173.71', '173.72', '173.79', '173.80', '173.81', '173.82', '173.91', '173.92', '173.99', '282.40', '282.43', '282.44', '282.46', '284.11', '284.12', '284.19', '286.52', '286.53', '286.59', '294.20', '294.21', '310.81', '310.89', '331.6', '348.82', '358.30', '365.70', '365.72', '365.73', '414.4', '415.13', '425.11', '425.18', '444.09', '488.81', '488.82', '488.89', '512.2', '512.82', '512.83', '512.84', '512.89', '516.31', '516.32', '516.33', '516.34', '516.35', '516.36', '516.37', '516.4', '516.5', '518.51', '518.52', '518.53', '539.01', '539.09', '539.81', '539.89', '573.5', '596.81', '596.82', '596.83', '596.89', '629.31', '649.81', '704.

In [5]:
splits = m4inpatient_dataset.random_splits(0.8, 0.9, 42)

In [6]:
list(map(len, splits))

[40736, 5092, 5092]

In [7]:
import jax
jax.config.update('jax_platform_name', 'cpu')

In [8]:
m4inaptients = Inpatients(m4inpatient_dataset, splits[2], max_workers=10)
# m4inaptients_jax = m4inaptients.to_jax_arrays()

DEBUG:root:Loading subjects..


INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


DEBUG:root:Extracting dx codes...
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                            ['041.41', '041.42', '041.43', '041.49', '173.00', '173.01', '173.02', '173.09', '173.10', '173.11', '173.12', '173.19', '173.20', '173.21', '173.22', '173.29', '173.30', '173.31', '173.32', '173.39']...
                            dx_icd10->dx_icd9 Unrecognised s_codes
                            (49910):
                            ['E08.3211', 'E08.3212', 'E08.3213', 'E08.3219', 'E08.3291', 'E08.3292', 'E08.3293', 'E08.3299', 'E08.3311', 'E08.3312', 'E08.3313', 'E08.3319', 'E08.3391', 'E08.3392', 'E08.3393', 'E08.3399', 'E08.3411', 'E08.3412', 'E08.3413', 'E08.3419']...
                            dx_icd10->dx_icd9 Unrecognised t_codes
                            (169):
                            ['041.41', '041.42', '041.43', '041.49', '173.00', '173.01', '173.02', '173.09', '173.10', '173.11', '173.12', '173.19', '173.

DEBUG:root:[DONE] Extracting procedures
DEBUG:root:Extracting inputs...


DEBUG:root:[DONE] Extracting inputs
DEBUG:root:Extracting observables...


DEBUG:root:[DONE] Extracting observables
DEBUG:root:[DONE] Loading subjects


In [14]:
m4inpatient_dataset.df['obs'].g

,hadm_id,time_bin,code,value
0,20000094,2150-03-02 15:00:00,o02,73.0
1,20000094,2150-03-02 22:00:00,o02,38.0
2,20000094,2150-03-03 03:00:00,o02,37.0
3,20000147,2121-08-30 17:00:00,o02,390.0
4,20000147,2121-08-30 19:00:00,o02,253.0
...,...,...,...,...
101972686,23874681,2149-10-07 19:00:00,o47,0.0
101972687,25880850,2156-02-16 18:00:00,o47,4.0
101972688,22838118,2128-02-08 08:00:00,o47,3.0
101972689,26054565,2150-12-18 01:00:00,o47,0.0


In [15]:
import seaborn as sns

DEBUG:matplotlib:matplotlib data path: /home/asem/GP/env/icenode-dev/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/home/asem/.config/matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', '_distutils_hack', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator', 'keyword', '_heapq', 'heapq', 'itertools', 'reprlib', '_collections', 'collections', '_functools', 'functools', 'conte

DEBUG:matplotlib:CACHEDIR=/home/asem/.cache/matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from /home/asem/.cache/matplotlib/fontlist-v330.json
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.
DEBUG:matplotlib.pyplot:Loaded backend module://matplotlib_inline.backend_inline version unknown.


In [17]:
m4inpatient_dataset.df['obs'].code.unique()

array(['o02', 'o12', 'o01', 'o07', 'o05', 'o00', 'o48', 'o04', 'o32',
       'o34', 'o09', 'o03', 'o33', 'o27', 'o22', 'o55', 'o13', 'o08',
       'o10', 'o11', 'o19', 'o06', 'o30', 'o29', 'o26', 'o35', 'o28',
       'o31', 'o36', 'o16', 'o15', 'o14', 'o59', 'o18', 'o24', 'o23',
       'o20', 'o21', 'o17', 'o25', 'o56', 'o58', 'o50', 'o49', 'o51',
       'o54', 'o53', 'o52', 'o57', 'o39', 'o40', 'o41', 'o38', 'o37',
       'o45', 'o44', 'o46', 'o43', 'o42', 'o47'], dtype=object)

In [ ]:
import matplotlib.pyplot as plt
from lib.ehr.coding_scheme import MIMIC4Observables
C = MIMIC4Observables()
for code in m4inpatient_dataset.df['obs'].code.unique():
    df = m4inpatient_dataset.df['obs']
    code_df = df[df.code == code]
    
    q1 = code_df.value.quantile(.05)
    q2 = code_df.value.quantile(.95)
    qin_mask = code_df.value.between(q1, q2)

    mu = code_df.value.mean()
    std = code_df.value.std()
    code_df['z'] = (code_df.value - mu) / std
    x1 = mu + 3 * std
    x2 = mu - 3 * std

    zin_mask = code_df.z.between(-3, 3)

    

    textstr = '\n'.join((
        f'n={len(code_df)}',
        f'(min, max): ({code_df.value.min():0.2f}, {code_df.value.max():0.2f})',
        f'p(qin): {qin_mask.mean():0.3f}',
        f'p(zin): {zin_mask.mean():0.3f}',
        f'p(zin or qin): {(qin_mask | zin_mask).mean(): 0.3f}'))
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    
    g = sns.displot(code_df, x="value", kde=True)

    for ax in g.axes.flat:
        args = zip([x1, x2, q1, q2], 
                   ["solid", "solid", "dashed", "dashed"], 
                   ["red", "red", "red", "red"])
        ax.set_title(C.desc[code])
            # place a text box in upper left in axes coords
        ax.text(0.05, 0.95, textstr, transform=ax.transAxes, fontsize=10,
                verticalalignment='top', bbox=props)
        for x, style, color in args:            
            ax.axvline(x=x, ymin = 0, ymax = 1.0, linestyle=style, color=color)
    g.savefig(f"obs_outlier_{C.desc[code]}.pdf", bbox_inches='tight')


In [35]:
g

In [ ]:
sns.displot(m4inpatient_dataset.df['obs'], x="value", hue="code", kde=True, multiple="stack")


In [9]:
m4inaptients.size_in_bytes / 1024 ** 3

1.0064665777608752

In [10]:
m4inaptients_jax = m4inaptients.to_jax_arrays(splits[2])

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.002241373062133789 sec
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'interpreter'
DEBUG:jax._src.lib.xla_bridge:Backend 'interpreter' initialized
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'cpu'
DEBUG:jax._src.lib.xla_bridge:Backend 'cpu' initialized
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'tpu_driver'
INFO:jax._src.lib.xla_bridge:Unable to initialize backend 'tpu_driver': NOT_FOUND: Unable to find driver in registry given worker: 
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'cuda'
DEBUG:jax._src.lib.xla_bridge:Backend 'cuda' initialized
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'rocm'
INFO:jax._src.lib.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: Interpreter Host CUDA
DEBUG:jax._src.lib.xla_bridge:Initializing backend 'tpu'
INFO:jax._src.lib.xla_bridge:Unable to 

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002789497375488281 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140363908240784) for with global shapes and types (ShapedArray(float32[2]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002713203430175781 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140363908240784) for with global shapes and types (ShapedArray(float32[2,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00022530555725097656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140363908240784) for with global shapes and types (ShapedArray(bool[2,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002605915069580078 sec
DEBUG:jax

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00028634071350097656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330437488) for with global shapes and types (ShapedArray(float32[60,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00022363662719726562 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330437488) for with global shapes and types (ShapedArray(bool[60,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00022292137145996094 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330437488) for with global shapes and types (ShapedArray(int32[17]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001971721649169922 sec
DEBUG:

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002593994140625 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140363908303744) for with global shapes and types (ShapedArray(float32[40,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00031447410583496094 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370484492384) for with global shapes and types (ShapedArray(bool[40,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00023674964904785156 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370484492384) for with global shapes and types (ShapedArray(float32[7]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00037670135498046875 sec
DEBUG:ja

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002129077911376953 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330513136) for with global shapes and types (ShapedArray(float32[6,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00019478797912597656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330513136) for with global shapes and types (ShapedArray(bool[6,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00024056434631347656 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370330462704) for with global shapes and types (ShapedArray(float32[69]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.000232696533203125 sec
DEBUG:ja

...

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002532005310058594 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140362384791808) for with global shapes and types (ShapedArray(float32[768,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00029087066650390625 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140362384791648) for with global shapes and types (ShapedArray(bool[768,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002570152282714844 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140362384793408) for with global shapes and types (ShapedArray(int32[428]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00024437904357910156 sec
DEBU

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00023865699768066406 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140365006668720) for with global shapes and types (ShapedArray(int32[787]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002086162567138672 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140365006671280) for with global shapes and types (ShapedArray(float32[787]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00025844573974609375 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140365951775744) for with global shapes and types (ShapedArray(int32[834]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00020885467529296875 sec
DEBUG:ja

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.000244140625 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370897769872) for with global shapes and types (ShapedArray(int32[515]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00029540061950683594 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370897769872) for with global shapes and types (ShapedArray(float32[515]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002460479736328125 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140363104745728) for with global shapes and types (ShapedArray(int32[1415]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00021576881408691406 sec
DEBUG:jax.inter

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002009868621826172 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370906854400) for with global shapes and types (ShapedArray(bool[520,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0003032684326171875 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370906787488) for with global shapes and types (ShapedArray(int32[549]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002448558807373047 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140370906785728) for with global shapes and types (ShapedArray(float32[549]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0002484321594238281 sec
DEBUG:jax

DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00025725364685058594 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140365592886944) for with global shapes and types (ShapedArray(float32[548,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0001888275146484375 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140365592884944) for with global shapes and types (ShapedArray(bool[548,60]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.0003249645233154297 sec
DEBUG:jax.interpreters.pxla:Compiling prim_fun (140367961101232) for with global shapes and types (ShapedArray(int32[3287]),). Argument mapping: (OpShardingSharding({replicated}),).
DEBUG:jax._src.dispatch:Finished tracing + transforming jit(convert_element_type) in 0.00020694732666015625 sec
DEB

In [12]:
m4inaptients_jax.size_in_bytes / 1024 ** 3

0.8186313798651099

In [ ]:
import jax.tree_util as jtree
import equinox as eqx
import jax.numpy as jnp
is_arr = eqx.filter(m4inaptients.subjects, eqx.is_array)
arr_size = jtree.tree_map(lambda a, flag: jnp.size(a) * jnp.dtype(a).itemsize if flag is not None else 0, 
                         m4inaptients.subjects, is_arr)
batch_size = sum(jtree.tree_leaves(arr_size))
batch_size
# is_arr = eqx.filter(m4inaptients.subjects, eqx.is_array)
# jtree.tree_leaves(m4inaptients.subjects)

In [ ]:
is_arr[0].admissions[0].procedures

In [ ]:
m4inaptients.subjects[0].admissions[0].procedures

In [ ]:
eqx.is_array([0, 1])

In [ ]:
import jax.numpy as jnp
a = jnp.array([1, 1, 0], dtype=jnp.float16)
jnp.dtype(a).itemsize * jnp.size(a)


In [ ]:
a.size

In [ ]:
from lib.ehr.coding_scheme import DxICD9, DxICD10

In [ ]:
s = DxICD10()

In [ ]:
len(s)

## TODO

1. Squeeze code vectors.
2. Squeeze float32 to float16.

In [ ]:
!ls  /home/asem/GP/ICE-NODE/lib/ehr/resources/

In [ ]:
df = pd.read_csv('/home/asem/GP/ICE-NODE/lib/ehr/resources/mimic4_int_grouper_proc.csv.gz')

In [ ]:
df.group.nunique()